In [1]:
import json
import pandas as pd
import requests

import portstress

from deribit import get_order_book, get_atm_ivol

pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [2]:
# Load stress parameters
file_path = './parameters/'

with open(file_path + 'spot_vol_shocks.json', 'r') as f:
    spot_vol_shocks_param = json.load(f)

crypto_parameters = portstress.core.utils.CryptoParameters(spot_vol_shocks_param)

with open(file_path + 'vol_surface_shocks.json', 'r') as f:
    vol_surface_shocks_param = json.load(f)

vol_surface_shocks_parameters = portstress.core.utils.VolSurfaceParameters(vol_surface_shocks_param)


In [30]:
# load position data
df = pd.read_excel('positions.xlsx')

# Splitting the 'instrument' column by "-"
df[['underlying','expiry','strike','put_call']] = df['instrument'].str.split('-', expand=True)
df['strike'] = df['strike'].astype(float)
df['put_call'] = df['put_call'].map({'P':'put','C':'call'})
df['underlying-expiry'] = df['underlying'] + "-" + df['expiry']
df['expiry'] = pd.to_datetime(df['expiry'], format='%d%b%y')
# Calculate time to expiry
valuation_day = pd.to_datetime('2024-08-05')
df['time_to_expiry'] = (df['expiry'] - valuation_day).dt.days / 365
# convert expiry to a string
df['expiry'] = df['expiry'].dt.strftime('%Y-%m-%d')


# get market data from deribit
deribit_res = get_order_book(instruments=df['instrument'].to_list())
df = pd.merge(df, deribit_res,how='left',left_on='instrument',right_on='instrument_name')


# calculate input for BSM		
df['cost_of_carry_rate'] = 'default'
df['multiplier'] = 1
df['rate'] = 0.03
df['vol'] = df['mark_iv'] / 100
df['spot'] = df['index_price'].values[0]
# df['position_vega'] = df['vega'] * df['quantity']

# calculate atm_ivol
df_atm = get_atm_ivol()
df = pd.merge(df,df_atm,on='underlying-expiry',how='left')


1912 instruments have been crawled.


In [3]:
df=pd.read_csv('temp2.csv')

In [34]:
df=df[['instrument','underlying','expiry','strike','put_call','multiplier',
    'quantity','time_to_expiry','rate','cost_of_carry_rate','vol','spot','atm_ivol']]

## Spot Vol Shocks

In [35]:
spot_vol_shocks = portstress.crypto.CryptoSpotVolShocks(crypto_parameters)

In [36]:
post_stress, col = spot_vol_shocks.apply_spot_vol_shocks(df)

In [37]:
portstress.core.black_scholes.bs_pricing(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')
portstress.core.black_scholes.calc_vega(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')

# np.float64(3606.873391368419)
# np.float64(3700.518323796012)

np.float64(93.56524889234046)

In [38]:
post_stress.head()

,instrument,underlying,expiry,strike,put_call,multiplier,quantity,time_to_expiry,rate,cost_of_carry_rate,vol,spot,atm_ivol,spot_shock,vol_shock,scenario_1,scenario_2
0,BTC-25OCT24-50000-P,BTC,2024-10-25,50000.0,put,1,-25.0,0.221918,0.03,default,0.5653,59504.4,55.80,0.15,0,24030.049555,31858.836608
1,BTC-25OCT24-84000-C,BTC,2024-10-25,84000.0,call,1,0.0,0.221918,0.03,default,0.6050,59504.4,55.80,0.15,0,0.000000,0.000000
2,BTC-27DEC24-105000-C,BTC,2024-12-27,105000.0,call,1,-8.8,0.394521,0.03,default,0.6713,59504.4,59.98,0.15,0,-8421.599219,-13755.021575
3,BTC-27DEC24-110000-C,BTC,2024-12-27,110000.0,call,1,-28.9,0.394521,0.03,default,0.6796,59504.4,59.98,0.15,0,-23988.652671,-39314.261939
4,BTC-27DEC24-115000-C,BTC,2024-12-27,115000.0,call,1,-1.7,0.394521,0.03,default,0.6877,59504.4,59.98,0.15,0,-1229.013553,-2020.437154


In [42]:
print(post_stress.scenario_1.sum())
print(post_stress.scenario_2.sum())


-297289.14819358406
-402718.54161190655


## Vol Surface Shocks

In [39]:
vol_surface_shocks = portstress.crypto.CryptoVolSurfaceShocks(vol_surface_shocks_parameters)

In [40]:
vol_surface_shocks.run(df)

[{'product': 'BTC',
  'measure': 'Concentration',
  'value': np.float64(-1044.9841259922352),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'TermStructure',
  'value': np.float64(-2152.89095353523),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'Skew',
  'value': np.float64(-1835.289621949277),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'BidAsk',
  'value': np.float64(-7824.9305832293185),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'Sum',
  'value': np.float64(-12858.095284706062),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'Sum',
  'measure': 'Concentration',
  'value': -1044.9841259922352,
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'Sum',
  'measure': 'TermStructure',
  'value': -2152.89095353523,
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'pr

## Ivol Timing

In [2]:
df = pd.read_excel('ivol_timing.xlsx')
df['time'] = pd.to_datetime(df['timestamp'], unit='ms')

In [28]:
# Volatility Index

from datetime import datetime, timedelta
import requests

ccy='BTC'
end_timestamp = int(datetime.now().timestamp() * 1000)
time_24_hours_ago = datetime.now() - timedelta(hours=24)
start_timestamp = int(time_24_hours_ago.timestamp() * 1000)
url = f"https://test.deribit.com/api/v2/public/get_volatility_index_data"

r = requests.get(url, params = {'currency': ccy,
                                'end_timestamp': end_timestamp,
                                'start_timestamp': start_timestamp,
                                'resolution':3600})

df_ivol_index = pd.DataFrame(r.json()['result']['data']).rename(columns={0:'timestamp',1:'open',2:'high',3:'low',4:'close'})
df_ivol_index['time'] = pd.to_datetime(df_ivol_index['timestamp'], unit='ms')
df_ivol_index['ivol_index'] = df_ivol_index['close']


In [29]:
import plotly.graph_objects as go

# Create separate traces for buy and sell
buy_trace = go.Scatter(x=df[df['direction'] == ' buy']['time'], y=df[df['direction'] == ' buy']['iv'],
                       mode='markers', marker=dict(color='blue', size=5), name='Buy')
sell_trace = go.Scatter(x=df[df['direction'] == ' sell']['time'], y=df[df['direction'] == ' sell']['iv'],
                        mode='markers', marker=dict(color='red', size=5), name='Sell')
# Line trace for ivol data
line_trace = go.Scatter(
    x=df_ivol_index['time'],
    y=df_ivol_index['ivol_index'],
    mode='lines',
    line=dict(color='green'),
    name='Ivol Index'
)

# Create figure with both traces
fig = go.Figure([buy_trace, sell_trace,line_trace])

# Update layout
fig.update_layout(title='IV vs Time',
                  xaxis_title='Time',
                  yaxis_title='IV')

# Show plot
fig.show()